<a href="https://colab.research.google.com/github/3000minsu/2023-LG-Aimers/blob/main/2_k_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
print('Done.')
print(test.shape)
print(train.shape)


Done.
(310, 2877)
(598, 2877)


In [ ]:
#그래서 하나의 세트에서 모두 결측치인 열은 두 세트에서 모두 삭제 
#data leakage 인가?

invalid=[]
for col in train.columns:
  if(train[col].isnull().all()):
    invalid+=[col]

train = pd.concat([train,test], ignore_index=True) #인덱스 초기화
train = train.drop(columns=invalid)
tmp=train
train = tmp.iloc[:598,:]
test = tmp.iloc[598:,:]

#진또배기 열들 리스트 저장장
column_set=[]
for col in train.columns:
  column_set+=[col]

print(test.shape)
print(train.shape)


(310, 2795)
(598, 2795)


In [ ]:

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5) 
train=imputer.fit_transform(train)
test = imputer.transform(test)

In [ ]:

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train)
train_scaled=ss.transform(train)
test_scaled=ss.transform(test)



In [ ]:
RF = RandomForestClassifier(random_state=37).fit(train, train_y)
print('Done.')

Done.


In [ ]:
preds = RF.predict(test)
print('Done.')

Done.


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
submit['Y_Class'] = preds

In [ ]:
submit.to_csv('./baseline_submission.csv', index=False)